In [4]:
# imports

from filterpy.kalman import KalmanFilter
from filterpy.kalman import predict
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

import numpy as np 
import csv 
import seaborn as sns
import pandas as pd 

In [ ]:
# load data

In [ ]:
# Create zs matrix: (n_steps, 3) where each row corresponds to measurements at one time step

zs = np.column_stack((basal_rate_data, heart_rate_data, respiratory_data))

In [ ]:
# Initial P matrix (state covariance matrix)

#creation of P matrix values 
unified_array = np.array([processed_basal_rate[650:], processed_heart_rate[650:], processed_respiratory[650:]])
P_threebythree = np.cov(unified_array)

final_P = np.array([
    [7, 0,            0,            0         ],
    [0, 98.18160966, -34.72900601, -1.22780453],
    [0, -34.72900601, 470.32652132, 3.1424907],
    [0, -1.22780453,  3.1424907,    3.33721444],
])

In [ ]:
# Initial state X

X = np.array([
    [97],
    [np.mean(processed_basal_rate[650:])],
    [np.mean(processed_heart_rate[650:])],
    [np.mean(processed_respiratory[650:])],
])

In [ ]:
# Process model matrix F

dt = 1  # 1 second time step
F = np.array([
    [1, 0, dt, 0.5*dt**2],
    [0, 1, 0, dt],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
])

In [ ]:
# Measurement noise covariance matrix R

R = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

In [ ]:
# Two options for Q (process noise covariance)

Q_filterpy = Q_discrete_white_noise(dim=4, dt=1., var=7)
Q_manual = np.array([
    [0.7, 0, 0, 0],
    [0, 9.818160966, 0, 0],
    [0, 0, 47.032652132, 0],
    [0, 0, 0, 0.333721444],
])

In [ ]:
# Measurement matrix H (maps state to measurement)

H = np.array([
    [0, 1, 0, 0],  # basal rate mapping
    [0, 0, 1, 0],  # heart rate mapping
    [0, 0, 0, 1]   # respiratory rate mapping
])

In [5]:
# initialize

def initialize_kalman_filter(X, P, R, Q, F, H):
    kf = KalmanFilter(dim_x=4, dim_z=3)
    kf.x = X
    kf.P = P
    kf.R = R
    kf.Q = Q
    kf.F = F
    kf.H = H
    return kf

ValueError: dim must be between 2 and 4

In [ ]:
# use filter

interpolated = list()
dates = list()

with open("interpolated_sleep_analysis.csv", 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        interpolated.append(float(row[1]))
        dates.append(pd.to_datetime(row[0]))

S = iter(interpolated)

xs = list()
xxs = list()
z = next(S, None)
while z is not None:

    kf.predict()
    kf.update(z)

    xs.append(kf.x_post[0])
    xxs.append(kf.x_post[1])

    z = next(S, None)

In [ ]:
# plot

sns.lineplot(
    x=range(len(xs)), y=xs
)